In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../pruebillas.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
import pandas as pd
import talib as ta
import pandas_ta as pandas_ta
import numpy as np
import matplotlib.pyplot as plt
from backtesting import Strategy
import plotly.express as px

from datetime import datetime
import MetaTrader5 as mt5
import pytz

from backbone.utils.general_purpose import diff_pips

In [3]:
def plot_stats(data, stats, strategy):
    equity_curve = stats._equity_curve
    aligned_data = data.reindex(equity_curve.index)
    bt = Backtest(aligned_data, strategy, cash=15_000, commission=0.002)
    print(stats)
    bt.plot(results=stats, resample=False)

In [24]:
def plot_full_equity_curve(data, stats_list, warmup_bars, lookback_bars, overlay_price=True):
    equity_curves = [x["_equity_curve"].iloc[warmup_bars:] for x in stats_list]

    combined = pd.Series(dtype=float)
    for curve in equity_curves:
        normalized_curve = curve["Equity"] / curve["Equity"].iloc[0]  # Normaliza la curva a su valor inicial
        if combined.empty:
            combined = normalized_curve
        else:
            # Alinea la nueva curva con la última de la serie combinada
            normalized_curve = normalized_curve * combined.iloc[-1]
            combined = pd.concat([combined, normalized_curve])

    fig = px.line(x=combined.index, y=combined)
    fig.update_traces(textposition="bottom right")
    fig.show()


In [5]:
# symbols_path = './backbone/data/backtest/symbols/USDCAD.csv'
# df = pd.read_csv(symbols_path)
# df = df[['Date','Open','High','Low','Close', 'Volume']]

print("MetaTrader5 package author: ", mt5.__author__)
print("MetaTrader5 package version: ", mt5.__version__)

# establish connection to MetaTrader 5 terminal
if not mt5.initialize():
    raise Exception("initialize() failed, error code =",mt5.last_error())

# set time zone to UTC
timezone = pytz.timezone("Etc/UTC")

# create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
utc_from = datetime(2024, 6, 1, tzinfo=timezone)
utc_to = datetime(2024, 8, 1, tzinfo=timezone)
rates = mt5.copy_rates_range('USDCAD', mt5.TIMEFRAME_M1
, utc_from, utc_to)

mt5.shutdown()

# create DataFrame out of the obtained data
df = pd.DataFrame(rates)

# convert time in seconds into the datetime format
df['time'] = pd.to_datetime(df['time'], unit='s')
                          
df = df.rename(columns={
  'time':'Date', 
  'open':'Open', 
  'high':'High', 
  'low':'Low', 
  'close':'Close', 
  'tick_volume':'Volume'
})

df

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288


,Date,Open,High,Low,Close,Volume,spread,real_volume
0,2024-06-03 00:00:00,1.36310,1.36310,1.36310,1.36310,1,30,0
1,2024-06-03 00:03:00,1.36284,1.36284,1.36242,1.36256,4,85,0
2,2024-06-03 00:05:00,1.36256,1.36256,1.36256,1.36256,1,116,0
3,2024-06-03 00:06:00,1.36271,1.36276,1.36271,1.36273,3,67,0
4,2024-06-03 00:07:00,1.36273,1.36273,1.36256,1.36256,2,70,0
...,...,...,...,...,...,...,...,...
61152,2024-07-31 23:56:00,1.38066,1.38066,1.38056,1.38063,53,22,0
61153,2024-07-31 23:57:00,1.38063,1.38063,1.38053,1.38060,35,22,0
61154,2024-07-31 23:58:00,1.38060,1.38078,1.38060,1.38073,39,22,0
61155,2024-07-31 23:59:00,1.38072,1.38072,1.38061,1.38070,28,23,0


In [6]:
train_start = '2024-06-01'
train_end = '2024-07-01'

test_start = '2024-07-01'
test_end = '2024-07-15'

train_data = df[(df.Date > train_start) & (df.Date < train_end)]
test_data = df[(df.Date > test_start) & (df.Date < test_end)]

train_data.loc[:, 'Date'] = pd.to_datetime(train_data.Date)
test_data.loc[:, 'Date'] = pd.to_datetime(test_data.Date)

train_data = train_data.set_index('Date')
test_data = test_data.set_index('Date')

df = df.set_index('Date')


# Bullish Engulfing Strategy

In [7]:

class BullishEngulfing(Strategy):
    position_size = 3500
    pip_size = 0.0001
    sl_pips = 10
    rr = 2

    def init(self):
        self.engulfing = self.I(ta.CDLENGULFING, self.data.Open, self.data.High, self.data.Low, self.data.Close)
        self.ema_200 = self.I(ta.EMA, self.data.Close, timeperiod=200)
        self.rsi = self.I(ta.RSI, self.data.Close, 14)

        # self.random = None

    def next(self):
        close_prices = self.data.Close
        actual_close_price = close_prices[-1]
       
        if self.position:
            pass
  
        else: 
            if actual_close_price > self.ema_200 and self.rsi > 55 and self.engulfing == 100:
                sl_price = actual_close_price - self.sl_pips * self.pip_size
                tp_price = actual_close_price + self.rr * self.sl_pips * self.pip_size
                self.buy(sl=sl_price, tp=tp_price)

            elif actual_close_price < self.ema_200 and self.rsi < 45 and self.engulfing == -100:
                sl_price = actual_close_price + self.sl_pips * self.pip_size
                tp_price = actual_close_price - self.rr * self.sl_pips * self.pip_size

                self.sell(sl=sl_price, tp=tp_price)


In [8]:

bt_train = Backtest(
    train_data, 
    BullishEngulfing, 
    cash=15000, 
    # commission=0.0002,
    margin=1/30
)

stats = bt_train.optimize(
    sl_pips=[6, 7, 8, 10],
    rr=[0.5, 0.8, 1],
    maximize='Return [%]',
    max_tries=1000,
)

print(f'sl pips: {stats._strategy.sl_pips}')
print(f'rr: {stats._strategy.rr}')

stats

  0%|          | 0/12 [00:00<?, ?it/s]

sl pips: 6
rr: 0.8


Start                     2024-06-03 00:00:00
End                       2024-06-28 23:58:00
Duration                     25 days 23:58:00
Exposure Time [%]                   38.541992
Equity Final [$]                   19048.0377
Equity Peak [$]                    21877.1451
Return [%]                          26.986918
Buy & Hold Return [%]                0.336732
Return (Ann.) [%]                   762.31001
Volatility (Ann.) [%]              748.153842
Sharpe Ratio                         1.018921
Sortino Ratio                       16.393323
Calmar Ratio                        33.040122
Max. Drawdown [%]                  -23.072252
Avg. Drawdown [%]                   -0.947929
Max. Drawdown Duration        8 days 07:29:00
Avg. Drawdown Duration        0 days 04:29:00
# Trades                                  222
Win Rate [%]                        59.009009
Best Trade [%]                       0.193474
Worst Trade [%]                     -0.048022
Avg. Trade [%]                    

In [9]:
bt_train.plot(filename='./RsiBBands.html', resample=False)

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p1656', ...)

In [10]:
bt_test = Backtest(
    test_data, 
    BullishEngulfing, 
    cash=15000, 
    # commission=0.0002,
    margin=1/30
)
 
test_stats = bt_test.run(
    sl_pips=stats._strategy.sl_pips,
    rr=stats._strategy.rr,
)



test_stats

Start                     2024-07-01 00:01:00
End                       2024-07-12 23:58:00
Duration                     11 days 23:57:00
Exposure Time [%]                   50.537713
Equity Final [$]                  13418.06095
Equity Peak [$]                   17174.02666
Return [%]                          -10.54626
Buy & Hold Return [%]               -0.279403
Return (Ann.) [%]                  -92.544188
Volatility (Ann.) [%]                6.304836
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -23.124488
Avg. Drawdown [%]                   -3.009258
Max. Drawdown Duration        9 days 06:50:00
Avg. Drawdown Duration        0 days 23:24:00
# Trades                                  108
Win Rate [%]                        47.222222
Best Trade [%]                       0.401282
Worst Trade [%]                     -0.048436
Avg. Trade [%]                    

In [11]:
bt_test.plot(filename='./ConsecutiveCandlesGridTest.html', resample=False)


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p2943', ...)

# WFO

In [17]:
def walk_forward(
        strategy,
        data_full,
        warmup_bars,
        lookback_bars=28*1440,
        validation_bars=7*1440,
        cash=15_000, 
        commission=0):

    stats_master = []

    for i in range(lookback_bars, len(data_full)-validation_bars, validation_bars):

        print(i)

        # To do anchored walk-forward, just set the first slice here to 0
        train_data = data_full.iloc[i-lookback_bars: i]

        bt_training = Backtest(train_data, strategy, cash=cash, commission=commission)
        stats_training = bt_training.optimize(
                sl_pips = [5, 8, 10],
                rr = [0.5, 1, 1.5, 2],
                maximize='Equity Final [$]')

        
        validation_data = data_full.iloc[i-warmup_bars:i+validation_bars]
        bt_validation = Backtest(validation_data, strategy, cash=cash, commission=commission)
        stats_validation = bt_validation.run(
                sl_pips = stats_training._strategy.sl_pips,
                rr = stats_training._strategy.rr)

        stats_master.append(stats_validation)

    return stats_master

In [18]:
import pickle


lookback_bars = 28*1440
validation_bars = 7*1440
warmup_bars = 14*60

if os.path.exists("stats.pickle"):
    with open("stats.pickle", "rb") as f:
        stats = pickle.load(f)
else:
    stats = walk_forward(BullishEngulfing, df, warmup_bars = warmup_bars)
    # with open("stats.pickle", "wb") as f:
    #     pickle.dump(stats, f)

40320


  0%|          | 0/12 [00:00<?, ?it/s]

50400


  0%|          | 0/12 [00:00<?, ?it/s]

In [30]:
plot_stats(df, stats[1], BullishEngulfing)

Start                     2024-07-19 22:37:00
End                       2024-07-31 12:43:00
Duration                     11 days 14:06:00
Exposure Time [%]                    70.14652
Equity Final [$]                 14778.040387
Equity Peak [$]                       15000.0
Return [%]                          -1.479731
Buy & Hold Return [%]                0.823672
Return (Ann.) [%]                  -34.125623
Volatility (Ann.) [%]                1.793635
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.508195
Avg. Drawdown [%]                   -1.508195
Max. Drawdown Duration        9 days 10:02:00
Avg. Drawdown Duration        9 days 10:02:00
# Trades                                   39
Win Rate [%]                        38.461538
Best Trade [%]                       0.068083
Worst Trade [%]                     -0.111442
Avg. Trade [%]                    

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single forma

In [25]:
plot_full_equity_curve(df, stats, warmup_bars = warmup_bars,
                       lookback_bars = lookback_bars, overlay_price = True)

In [17]:
trades = test_stats._trades.groupby(by=['ExitTime']).agg({'PnL':['sum','count'], 'Duration':'max'})
trades.columns = trades.columns.droplevel(0)
trades = trades.reset_index().rename(columns={'count':'ammount_trades'})
trades = trades.rename(columns={'sum':'profit'})
trades = trades.rename(columns={'max':'minutes_in_trade'})
trades

,ExitTime,profit,ammount_trades,minutes_in_trade
0,2024-07-01 04:26:00,161.27713,1,0 days 00:06:00
1,2024-07-01 04:49:00,-206.35708,1,0 days 00:13:00
2,2024-07-01 08:52:00,-193.58018,1,0 days 02:21:00
3,2024-07-01 10:06:00,-197.64976,1,0 days 01:00:00
4,2024-07-01 10:35:00,150.17064,1,0 days 00:14:00
...,...,...,...,...
103,2024-07-12 15:26:00,134.73952,1,0 days 01:03:00
104,2024-07-12 17:23:00,-195.18906,1,0 days 00:11:00
105,2024-07-12 18:02:00,139.89600,1,0 days 00:30:00
106,2024-07-12 18:40:00,126.72530,1,0 days 00:30:00


In [18]:
trades.minutes_in_trade.describe()


count                          108
mean     0 days 01:09:38.333333333
std      0 days 01:43:26.515795336
min                0 days 00:01:00
25%                0 days 00:14:00
50%                0 days 00:30:30
75%                0 days 01:21:45
max                0 days 12:04:00
Name: minutes_in_trade, dtype: object